In [1]:
import tensorflow as tf
import numpy as np
import dlsr

dlsr.helpers.config(True, True)

In [2]:
# Functions to calculate SSIM and PSNR

def round(x):
    return "%.2f" % np.round(x, 2)


def SSIM(a, b):
    return tf.image.ssim(a, b, 255).numpy()


def PSNR(a, b):
    return tf.image.psnr(a, b, 255).numpy()

In [3]:
# load set 14 dataset
set14 = dlsr.data.Set14(scale=4)
lr = set14.dataset("LR").batch(1)
hr = set14.dataset("HR").batch(1)

upscaled = {}

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-12 19:55:10.647459: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-12 19:55:10.647480: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/ognjenbjeletic/.pyenv/versions/3.10.9/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [4]:
# Upscale with Interploation-Based methods

lr_transformed = [np.uint8(x)[0] for x in lr]
new_size = [[np.shape(x)[0] * 4, np.shape(x)[1] * 4] for x in lr_transformed]

algs = [
    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    tf.image.ResizeMethod.BILINEAR,
    tf.image.ResizeMethod.BICUBIC,
    tf.image.ResizeMethod.LANCZOS3,
]

for alg in algs:
    upscaled[alg] = [
        tf.image.resize(
        image,
        size,
        method=alg
    ) for image, size in zip(lr_transformed, new_size)]
    

2023-02-12 19:55:10.710391: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [5]:
# Upscale with SRResNet 4x

model = tf.keras.models.load_model(
    "./results/saved-models/srres4x.h5",
)

upscaled["srres4x"] = [
    x.numpy() for x in
    dlsr.upscale(model, lr)
]
# free memory
del model

14/14 [==============================] - 3s 229ms/step


In [6]:
# SRGAN 4x

# Load the model from JSON
json_file = open('./results/saved-models/srgan4x/architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = tf.keras.models.model_from_json(loaded_model_json)

# Load the weights
model.load_weights("./results/saved-models/srgan4x/generator.h5")

upscaled["srgan4x"] = [
    x.numpy() for x in
    dlsr.upscale(model, lr)
]

# free memory
del model

14/14 [==============================] - 3s 218ms/step


In [7]:
# save LR images
for i, x in enumerate(lr):
    tf.keras.utils.save_img(path=f"results/set14/lr/image{i}.png", x=x[0], file_format="png")

# save HR images
for i, x in enumerate(hr):
    tf.keras.utils.save_img(path=f"results/set14/hr/image{i}.png", x=x[0], file_format="png")

# save upscaled images
for key in upscaled.keys():
    for i, x in enumerate(upscaled[key]):
        tf.keras.utils.save_img(
            path=f"results/set14/{key}/image{i}.png", x=x, file_format="png"
        )

# create dict for SSIM/PSNR
results = {}
for key in upscaled.keys():
    results[key] = {
       "ssim": 0,
        "psnr": 0
    }

# calculate SSIM/PSNR
for i, x in enumerate(hr):
    real = np.uint8(x)[0]
    print("-------------------------------------")
    for j, key in enumerate(upscaled.keys()):
        fake = np.uint8(upscaled[key][i])
        ssim = SSIM(real, fake)
        psnr = PSNR(real, fake)
        print(
            "| {:<4} | {:<10} | {:<4} = {:<6} |".format(
                f"img{i}" if j == 0 else "", f"{key}", "SSIM", round(ssim)
            )
        )
        print("| {:<4} | {:<10} | {:<4} = {:<6} |".format("", "", "PSNR", round(psnr)))
        results[key]["ssim"] += ssim
        results[key]["psnr"] += psnr

print("-------------------------------------")

-------------------------------------
| img0 | nearest    | SSIM = 0.37   |
|      |            | PSNR = 19.90  |
|      | bilinear   | SSIM = 0.36   |
|      |            | PSNR = 20.09  |
|      | bicubic    | SSIM = 0.39   |
|      |            | PSNR = 20.29  |
|      | lanczos3   | SSIM = 0.40   |
|      |            | PSNR = 20.34  |
|      | srres4x    | SSIM = 0.46   |
|      |            | PSNR = 20.77  |
|      | srgan4x    | SSIM = 0.39   |
|      |            | PSNR = 19.49  |
-------------------------------------
| img1 | nearest    | SSIM = 0.60   |
|      |            | PSNR = 22.87  |
|      | bilinear   | SSIM = 0.63   |
|      |            | PSNR = 23.25  |
|      | bicubic    | SSIM = 0.65   |
|      |            | PSNR = 23.58  |
|      | lanczos3   | SSIM = 0.66   |
|      |            | PSNR = 23.47  |
|      | srres4x    | SSIM = 0.71   |
|      |            | PSNR = 24.46  |
|      | srgan4x    | SSIM = 0.67   |
|      |            | PSNR = 23.52  |
------------

In [8]:
# print average SSIM/PSNR
print("------------------------------")
for key in results.keys():
    ssim = round(results[key]["ssim"] / 14)
    psnr = round(results[key]["psnr"] / 14)
    print(
        "| {:<10} | {:<4} = {:<6} |".format(
            f"{key}", "SSIM", ssim
        )
    )
    print(
        "| {:<10} | {:<4} = {:<6} |".format("", "PSNR", psnr)
    )
    print("------------------------------")

------------------------------
| nearest    | SSIM = 0.62   |
|            | PSNR = 22.98  |
------------------------------
| bilinear   | SSIM = 0.64   |
|            | PSNR = 23.71  |
------------------------------
| bicubic    | SSIM = 0.65   |
|            | PSNR = 21.93  |
------------------------------
| lanczos3   | SSIM = 0.65   |
|            | PSNR = 21.44  |
------------------------------
| srres4x    | SSIM = 0.73   |
|            | PSNR = 26.11  |
------------------------------
| srgan4x    | SSIM = 0.67   |
|            | PSNR = 24.30  |
------------------------------
